<a href="https://colab.research.google.com/github/thomngt/BigData_Project/blob/main/crawl_amazon_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# Header to set the requests as a browser requests
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

# URL of The amazon Review page
reviews_url = 'https://www.amazon.com/product-reviews/B01N1XXT7F/ref=cm_cr_arp_d_viewopt_sr?ie=UTF8&filterByStar=all_stars&reviewerType=all_reviews'

# Define Page No
len_page = 800


# Extra Data as Html object from amazon Review page
def reviewsHtml(url, len_page):

    # Empty List define to store all pages html data
    soups = []


    for page_no in range(1, len_page + 1):

        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }

        # Request make for each page
        response = requests.get(url, headers=headers)

        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')

        # Add single Html page data in master soups list
        soups.append(soup)

    return soups

# Grab Reviews name, description, date, stars, title from HTML
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []

    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')

    # Iterate all Reviews BOX
    for box in boxes:

        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        # create Dictionary with al review data
        data_dict = {
            'Name' : name,
            'Rating' : stars,
            'Title' : title,
            'Date' : date,
            'Review_Text' : description
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)

    return data_dicts

# Grab all HTML
html_datas = reviewsHtml(reviews_url, len_page)

# Empty List to Hold all reviews data
reviews = []

# Iterate all Html page
for html_data in html_datas:

    # Grab review data
    review = getReviews(html_data)

    # add review data in reviews empty list
    reviews += review

# Create a dataframe with reviews Data
df_reviews = pd.DataFrame(reviews)

print(df_reviews)

# Save data
df_reviews.to_csv('amazon_reviews.csv', index=False)

                       Name Rating  \
0                    Brdove    5.0   
1           Shariante Hawes    5.0   
2                      Ebie    4.0   
3                   DizNerd    1.0   
4                       kim    5.0   
...                     ...    ...   
1395              CA newbie    3.0   
1396                Heather    5.0   
1397             Tena Clark    5.0   
1398               Sophie26    4.0   
1399  Ivory Angel Alexander    4.0   

                                                  Title        Date  \
0     5.0 out of 5 stars\nVery roomy and comfortable...  05/02/2024   
1     5.0 out of 5 stars\nComfortable and great qual...  28/02/2024   
2              4.0 out of 5 stars\nExactly as described  27/02/2024   
3             1.0 out of 5 stars\nNot so slip resistant  10/10/2023   
4                      5.0 out of 5 stars\nSturdy shoes  23/02/2024   
...                                                 ...         ...   
1395         3.0 out of 5 stars\ntoo narrow at

In [ ]:
https://www.amazon.com/product-reviews/B08RTG3RN9/ref=acr_dp_hist_5?ie=UTF8&filterByStar=five_star&reviewerType=all_reviews